In [1]:
import sys
# enable importing the modules from probcalkge
sys.path.append('../')
sys.path.append('../probcalkge')

import numpy as np
import pandas as pd

from probcalkge import Experiment, ExperimentResult
from probcalkge import get_calibrators
from probcalkge import get_datasets,  get_kgemodels
from probcalkge import brier_score, negative_log_loss, ks_error, ece

In [2]:
cals = get_calibrators()
kges = get_kgemodels()
ds = get_datasets()

WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [3]:
exp = Experiment(
    cals=[cals.uncal, cals.platt, cals.isot, ], 
    datasets=[ds.fb13, ds.wn18, ds.yago39, ds.dp50, ds.kinship, ds.umls], 
    kges=[kges.transE, kges.complEx, kges.distMult, kges.hoLE], 
    metrics=[ece]
    )

In [4]:
exp.load_trained_kges('../saved_models/')

Loaded models:
{'DBpedia50': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000002134F864908>,
               'DistMult': <ampligraph.latent_features.models.DistMult.DistMult object at 0x00000213008DB048>,
               'HolE': <ampligraph.latent_features.models.HolE.HolE object at 0x0000021303F9FFC8>,
               'TransE': <ampligraph.latent_features.models.TransE.TransE object at 0x0000021307CA4048>},
 'FB13k': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x00000213005C6D88>,
           'DistMult': <ampligraph.latent_features.models.DistMult.DistMult object at 0x0000021304185208>,
           'HolE': <ampligraph.latent_features.models.HolE.HolE object at 0x0000021379817D48>,
           'TransE': <ampligraph.latent_features.models.TransE.TransE object at 0x0000021379777508>},
 'Kinship': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x0000021303F9F948>,
             'DistMult': <ampligraph.latent_fea

In [5]:
exp_res = exp.run_with_trained_kges()

training various calibrators for TransE on FB13k ...
training various calibrators for ComplEx on FB13k ...
training various calibrators for DistMult on FB13k ...
training various calibrators for HolE on FB13k ...
training various calibrators for TransE on WN11 ...
training various calibrators for ComplEx on WN11 ...
training various calibrators for DistMult on WN11 ...
training various calibrators for HolE on WN11 ...
training various calibrators for TransE on YAGO39 ...
training various calibrators for ComplEx on YAGO39 ...
training various calibrators for DistMult on YAGO39 ...
training various calibrators for HolE on YAGO39 ...
training various calibrators for TransE on DBpedia50 ...
training various calibrators for ComplEx on DBpedia50 ...
training various calibrators for DistMult on DBpedia50 ...
training various calibrators for HolE on DBpedia50 ...
training various calibrators for TransE on Nations ...
training various calibrators for ComplEx on Nations ...
training various cali

FB13k TransE UncalCalibrator 0.4999789314006405
FB13k TransE PlattCalibrator 0.6719829765717175
FB13k TransE IsotonicCalibrator 0.6721936625653127
FB13k ComplEx UncalCalibrator 0.5538934771616383
FB13k ComplEx PlattCalibrator 0.6476908815101972
FB13k ComplEx IsotonicCalibrator 0.6927987527389179
FB13k DistMult UncalCalibrator 0.5748356649249958
FB13k DistMult PlattCalibrator 0.6133912017529075
FB13k DistMult IsotonicCalibrator 0.6421077026799258
FB13k HolE UncalCalibrator 0.5574119332546772
FB13k HolE PlattCalibrator 0.4878644867689196
FB13k HolE IsotonicCalibrator 0.6391159615708748
WN11 TransE UncalCalibrator 0.5074596569572719
WN11 TransE PlattCalibrator 0.882624581345783
WN11 TransE IsotonicCalibrator 0.8812544402719984
WN11 ComplEx UncalCalibrator 0.5592205419669136
WN11 ComplEx PlattCalibrator 0.5898711052471328
WN11 ComplEx IsotonicCalibrator 0.6226529990865726
WN11 DistMult UncalCalibrator 0.566020501370141
WN11 DistMult PlattCalibrator 0.6024561047396731
WN11 DistMult Isotonic